In [1]:
import pandas as pd
import numpy as np
from skimage import io, transform

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset
from torchvision import transforms, utils, models
import torchvision as tv
from torchvision.transforms import v2

import lightning as L
import torchmetrics as tm
import torch.nn.functional as F

from IPython.display import clear_output

import os

In [2]:
L.seed_everything(42)

[rank: 0] Seed set to 42


42

### Creating the dataset and dataloader from `intel-image-classification`

In [3]:
# convert PIL image into torch Tensor then does specified transforms from docs: 
# https://pytorch.org/vision/main/models/generated/torchvision.models.resnet50.html
ds_transforms = v2.Compose([
    v2.ToImage(),
    v2.Resize(256),
    v2.CenterCrop(224),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [4]:
test_dataset = tv.datasets.ImageFolder("intel-image-classification/seg_test/seg_test", transform=ds_transforms)

In [5]:
test_dataset

Dataset ImageFolder
    Number of datapoints: 3000
    Root location: intel-image-classification/seg_test/seg_test
    StandardTransform
Transform: Compose(
                 ToImage()
                 Resize(size=[256], interpolation=InterpolationMode.BILINEAR, antialias=True)
                 CenterCrop(size=(224, 224))
                 ToDtype(scale=True)
                 Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], inplace=False)
           )

In [6]:
test_dataset, val_dataset = random_split(test_dataset, [.5, .5], generator=torch.Generator().manual_seed(42))

---
## Create Classifier Class and Feature Extractors

In [21]:
feature_extractors = {}

In [22]:
# create resnet50 feature extractor
resnet50 = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
resnet50.eval()
resnet50_backbone = list(resnet50.children())[:-1]
resnet50_feat_extractor = nn.Sequential(*resnet50_backbone)
feature_extractors['resnet50'] = resnet50_feat_extractor
clear_output()

In [23]:
class IntelClassifier(L.LightningModule):
    def __init__(self, feature_extractor_name, output_features, num_classes, classifier=None, optimizer=torch.optim.Adam, lr=1e-2):
        super().__init__()
        self.save_hyperparameters()
        if feature_extractor_name not in feature_extractors:
            raise ValueError(f"`feature_extractor_name` argument is invalid (should be one of {list(feature_extractors.keys())})")
        self.feature_extractor = feature_extractors[feature_extractor_name]
        self.classifier = classifier if classifier else nn.Sequential( # classifier layers after the feature extraction
            nn.Linear(output_features, 512),
            nn.LeakyReLU(),
            nn.Dropout(.2),
            nn.Linear(512, num_classes)
        )
        self.accuracy = tm.classification.Accuracy(task="multiclass", num_classes=num_classes)
        self.optimizer = optimizer
        self.lr = lr

    def forward(self, x):
        self.feature_extractor.eval()
        with torch.no_grad():
            features = self.feature_extractor(x).flatten(1)
        return self.classifier(features)

    def _batch_step(self, batch, batch_kind):
        if batch_kind == 'train':
            self.classifier.train()
        else:
            self.classifier.eval()
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = self.accuracy(y_hat, y)
        # logging onto tensorboard
        self.log(f"{batch_kind}_loss", loss, prog_bar=True)
        self.log(f"{batch_kind}_acc_f1", acc, prog_bar=True)
        return loss

    def training_step(self, batch, batch_idx):
        return self._batch_step(batch, 'train')

    def validation_step(self, batch, batch_idx):
        return self._batch_step(batch, 'val')

    def test_step(self, batch, batch_idx):
        return self._batch_step(batch, 'test')

    def predict_step(self, batch, batch_idx):
        self.eval()
        x, _ = batch
        return self(x)
    
    def configure_optimizers(self):
        optimizer = self.optimizer(self.parameters(), lr=self.lr)
        return optimizer

---
### Evaluating models

In [7]:
torch.set_float32_matmul_precision('high')

In [9]:
BATCH_SIZE=64

In [10]:
test_loader = DataLoader(test_dataset, BATCH_SIZE, num_workers=4)
val_loader = DataLoader(val_dataset, BATCH_SIZE, num_workers=4)

In [17]:
CKPT_PATH = 'resnet50-intel-raw-fulltrain.ckpt'

In [18]:
y = np.array([test_dataset[i][1] for i in range(len(test_dataset))])

In [19]:
loaded_model = IntelClassifier.load_from_checkpoint(checkpoint_path=CKPT_PATH)
# loaded_model = ResnetClassifier(50)
# checkpoint = torch.load(CKPT_PATH)
# loaded_model.load_state_dict(checkpoint["state_dict"])

In [20]:
trainer = L.Trainer(callbacks=[L.pytorch.callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=5)], max_epochs=50)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [22]:
trainer.test(loaded_model, val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
SLURM auto-requeueing enabled. Setting signal handlers.


Testing: |                                                | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       test_acc_f1          0.9300000071525574
        test_loss           0.18530550599098206
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.18530550599098206, 'test_acc_f1': 0.9300000071525574}]

In [23]:
trainer.test(loaded_model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
SLURM auto-requeueing enabled. Setting signal handlers.


Testing: |                                                | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       test_acc_f1          0.9313333034515381
        test_loss           0.17001409828662872
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.17001409828662872, 'test_acc_f1': 0.9313333034515381}]

---

### Automating Evaluation

In [24]:
# !mamba install scikit-learn -y

In [11]:
torch.set_float32_matmul_precision('high')

In [12]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

In [13]:
BATCH_SIZE = 64
test_loader = DataLoader(test_dataset, BATCH_SIZE, num_workers=4)
val_loader = DataLoader(val_dataset, BATCH_SIZE, num_workers=4)

In [14]:
# y = test_dataset.targets
y = np.array([test_dataset[i][1] for i in range(len(test_dataset))])
# y_train = train_whole_dataset.targets
y_val = np.array([val_dataset[i][1] for i in range(len(val_dataset))])

In [27]:
def get_test_preds(loaded_model, test_loader):
    trainer = L.Trainer()
    loaded_model.freeze()

    predictions_list = trainer.predict(loaded_model, test_loader) # 30-len list of 32 x 20 tensors
    predictions = torch.vstack(predictions_list).numpy() # 952 x 20
    top_preds = predictions.argmax(axis=1).flatten()

    return top_preds, predictions

def top_preds(all_predictions):
    return (
        np.argsort(all_predictions, axis=1)[:, -5:],
        np.argsort(all_predictions, axis=1)[:, -3:]
    )

def get_topk_accuracy(top_preds, ground_truths):
    ground_truths = np.array(ground_truths)
    #check if ground truth class lies somewhere in the top k
    #check if any of the top 5 predicted classes match the ground truth class
    # print(top_preds.shape)
    ground_truths = ground_truths.reshape(-1, 1)
    matches = np.any(top_preds == ground_truths, axis=1)

    # Count the number of matches
    num_matches = np.sum(matches)
    # print(num_matches)

    # Calculate the percentage of images where at least one of the top 5 predictions matches the ground truth
    percentage_matches = (num_matches / top_preds.shape[0]) * 100
    return percentage_matches

def performance_metrics(predictions, ground_truth, metric_type="Test"):
    accuracy = accuracy_score(ground_truth, predictions)
    recall = recall_score(ground_truth, predictions, average='weighted')
    precision = precision_score(ground_truth, predictions, average='weighted')
    f1 = f1_score(ground_truth, predictions, average='weighted')

    # print(f"{metric_type} Accuracy: {accuracy}")
    print(f"{metric_type} Recall: {recall}")
    print(f"{metric_type} Precision: {precision}")
    print(f"{metric_type} F1 Score: {f1}")
    return {'recall': recall, 'precision': precision, 'f1': f1}

In [59]:
fractions = ["full", "1_20", "1_50", "1_100"]
aug_types = ["raw", "tfms", "seg", "canny", "canny2"]

In [60]:
CKPTS = [f"resnet50-intel-{aug_type}-{fraction}train.ckpt" for aug_type in aug_types for fraction in fractions]

In [61]:
CKPTS

['resnet50-intel-raw-fulltrain.ckpt',
 'resnet50-intel-raw-1_20train.ckpt',
 'resnet50-intel-raw-1_50train.ckpt',
 'resnet50-intel-raw-1_100train.ckpt',
 'resnet50-intel-tfms-fulltrain.ckpt',
 'resnet50-intel-tfms-1_20train.ckpt',
 'resnet50-intel-tfms-1_50train.ckpt',
 'resnet50-intel-tfms-1_100train.ckpt',
 'resnet50-intel-seg-fulltrain.ckpt',
 'resnet50-intel-seg-1_20train.ckpt',
 'resnet50-intel-seg-1_50train.ckpt',
 'resnet50-intel-seg-1_100train.ckpt',
 'resnet50-intel-canny-fulltrain.ckpt',
 'resnet50-intel-canny-1_20train.ckpt',
 'resnet50-intel-canny-1_50train.ckpt',
 'resnet50-intel-canny-1_100train.ckpt',
 'resnet50-intel-canny2-fulltrain.ckpt',
 'resnet50-intel-canny2-1_20train.ckpt',
 'resnet50-intel-canny2-1_50train.ckpt',
 'resnet50-intel-canny2-1_100train.ckpt']

In [62]:
import logging
logging.getLogger("lightning.pytorch.utilities.rank_zero").setLevel(logging.WARNING)
logging.getLogger("lightning.pytorch.accelerators.cuda").setLevel(logging.WARNING)

In [63]:
import pandas as pd

In [64]:
test_results = {}
for fraction in fractions:
    for aug_type in aug_types:
        ckpt = f"resnet50-intel-{aug_type}-{fraction}train.ckpt"
        print(f"---{ckpt}---")
        loaded_model = IntelClassifier.load_from_checkpoint(checkpoint_path=ckpt)
        resnet_pred, resnet_all_pred = get_test_preds(loaded_model, test_loader)
        
        resnet_top5, resnet_top3 = top_preds(resnet_all_pred)
        
        perf_dict = performance_metrics(resnet_pred, y)
        test_results[(fraction, aug_type)] = perf_dict
        print()

---resnet50-intel-raw-fulltrain.ckpt---


SLURM auto-requeueing enabled. Setting signal handlers.


Predicting: |                                                                                        | 0/? [00…

Test Recall: 0.9313333333333333
Test Precision: 0.9312672755513962
Test F1 Score: 0.9312397885801864

---resnet50-intel-tfms-fulltrain.ckpt---


SLURM auto-requeueing enabled. Setting signal handlers.


...

Predicting: |                                                                                        | 0/? [00…

Test Recall: 0.854
Test Precision: 0.8628603039206826
Test F1 Score: 0.8538442882081906

---resnet50-intel-canny2-1_100train.ckpt---


SLURM auto-requeueing enabled. Setting signal handlers.


Predicting: |                                                                                        | 0/? [00…

Test Recall: 0.814
Test Precision: 0.8446055001806013
Test F1 Score: 0.8113269372246191



In [65]:
df = pd.DataFrame.from_dict(test_results, orient='index')

In [66]:
df

recall  precision        f1
full  raw     0.931333   0.931267  0.931240
      tfms    0.929333   0.929317  0.929205
      seg     0.916667   0.917359  0.916656
      canny   0.928667   0.928405  0.928437
      canny2  0.930000   0.929800  0.929832
1_20  raw     0.889333   0.890503  0.889000
      tfms    0.886000   0.890226  0.884921
      seg     0.864667   0.883181  0.860244
      canny   0.893333   0.898284  0.892508
      canny2  0.874667   0.889629  0.870913
1_50  raw     0.841333   0.855506  0.841207
      tfms    0.812000   0.840100  0.806699
      seg     0.826667   0.856083  0.817752
      canny   0.861333   0.873686  0.858688
      canny2  0.860667   0.873648  0.857548
1_100 raw     0.793333   0.825094  0.781761
      tfms    0.784000   0.823389  0.771427
      seg     0.846000   0.855312  0.845075
      canny   0.854000   0.862860  0.853844
      canny2  0.814000   0.844606  0.811327

In [67]:
df.to_csv('test_results.csv')

In [68]:
val_results = {}
for aug_type in aug_types:
    for fraction in fractions:
        ckpt = f"resnet50-intel-{aug_type}-{fraction}train.ckpt"
        print(f"---{ckpt}---")
        loaded_model = IntelClassifier.load_from_checkpoint(checkpoint_path=ckpt)
        resnet_pred, resnet_all_pred = get_test_preds(loaded_model, val_loader)
        
        resnet_top5, resnet_top3 = top_preds(resnet_all_pred)
        
        perf_dict = performance_metrics(resnet_pred, y_val, metric_type="Val")
        val_results[(fraction, aug_type)] = perf_dict
        print()

---resnet50-intel-raw-fulltrain.ckpt---


SLURM auto-requeueing enabled. Setting signal handlers.


Predicting: |                                                                                        | 0/? [00…

Val Recall: 0.9293333333333333
Val Precision: 0.929198505463445
Val F1 Score: 0.9291724728471301

---resnet50-intel-raw-1_20train.ckpt---


SLURM auto-requeueing enabled. Setting signal handlers.


...

Predicting: |                                                                                        | 0/? [00…

Val Recall: 0.858
Val Precision: 0.8705849557511752
Val F1 Score: 0.8557242368797233

---resnet50-intel-canny2-1_100train.ckpt---


SLURM auto-requeueing enabled. Setting signal handlers.


Predicting: |                                                                                        | 0/? [00…

Val Recall: 0.822
Val Precision: 0.8406897280185138
Val F1 Score: 0.8183323193091763



In [69]:
val_df = pd.DataFrame.from_dict(test_results, orient='index')

In [70]:
val_df

recall  precision        f1
full  raw     0.931333   0.931267  0.931240
      tfms    0.929333   0.929317  0.929205
      seg     0.916667   0.917359  0.916656
      canny   0.928667   0.928405  0.928437
      canny2  0.930000   0.929800  0.929832
1_20  raw     0.889333   0.890503  0.889000
      tfms    0.886000   0.890226  0.884921
      seg     0.864667   0.883181  0.860244
      canny   0.893333   0.898284  0.892508
      canny2  0.874667   0.889629  0.870913
1_50  raw     0.841333   0.855506  0.841207
      tfms    0.812000   0.840100  0.806699
      seg     0.826667   0.856083  0.817752
      canny   0.861333   0.873686  0.858688
      canny2  0.860667   0.873648  0.857548
1_100 raw     0.793333   0.825094  0.781761
      tfms    0.784000   0.823389  0.771427
      seg     0.846000   0.855312  0.845075
      canny   0.854000   0.862860  0.853844
      canny2  0.814000   0.844606  0.811327

In [71]:
val_df.to_csv('val_results.csv')